In [13]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np

In [16]:
mi_file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx'

In [46]:
df = pd.read_excel(mi_file_path, sheet_name = 'Deferrals')
df_df = df.dropna(how='all').dropna(how='all', axis=1)

df_df.columns = df_df.iloc[0]

df_df = df_df[1:].reset_index(drop=True)

In [47]:
display(df_df.head())

1,Act No,Act Name,2024.0,2025.0,Grand Total
0,3524,Deferred Excess of Loss Cover c/fwd,1247667.28,4163799.79,5411467.07
1,3532,Reinsurers' Provision for Unearned Premiums - QS,4413421.05,16497285.31,20910706.36
2,4852,Provision unearned premium,-6307147.26,-23567550.47,-29874697.73
3,4152,Deferred Profit Commission BS,0.00,0.00,0


In [41]:
def clean_financial_sheet(file_path, sheet_name):
    """
    Cleans a financial sheet (PL or BS) with a complex, multi-level header.
    This function dynamically adapts to the header structure of the specified sheet.
    Args:
        file_path (str): The path to the Excel file.
        sheet_name (str): The name of the sheet to clean ('PL' or 'BS').
    Returns:
        pandas.DataFrame: A cleaned DataFrame for the specified sheet.
    """
    # 1. Read the two header rows from the specified sheet
    header_df = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=None,
        skiprows=4,
        nrows=2
    )
    
    # 2. Construct the multi-level header
    top_header_raw = header_df.iloc[0]
    bottom_header = header_df.iloc[1].tolist()

    # The FutureWaring originates from this ffill() call. It is informational
    # and can be safely ignored in this context.
    top_header = top_header_raw.ffill().tolist()

    try:
        prior_year_index = bottom_header.index('Prior_Year')
        for i in range(prior_year_index, len(top_header)):
            top_header[i] = ''
    except ValueError:
        pass
        
    header_tuples = [
        (str(top) if pd.notna(top) else '', str(bot) if pd.notna(bot) else '')
        for top, bot in zip(top_header, bottom_header)
    ]
    
    if len(header_tuples) > 1:
        header_tuples[0] = ('', 'Code')
        header_tuples[1] = ('', 'Description')

    multi_header = pd.MultiIndex.from_tuples(header_tuples)
    
    # 3. Read the main data body using the original number of columns from the header
    num_cols = len(header_df.columns)
    data = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=None,
        skiprows=7,
        usecols=range(num_cols)
    )

    # 4. Combine data and headers and perform final cleanup
    data.columns = multi_header
    
    # Drop fully empty columns AND rows from the final combined dataframe
    data.dropna(how='all', axis=1, inplace=True)
    data.dropna(how='all', axis=0, inplace=True)
    data.reset_index(drop=True, inplace=True)

    return data

In [48]:
pl_df = clean_financial_sheet(file_path = mi_file_path, sheet_name='PL')
bs_df = clean_financial_sheet(file_path = mi_file_path, sheet_name='BS')

# Display the results for the PL Sheet
print("\n--- PL DataFrame Columns ---")
print(pl_df.columns)
print("\n--- PL DataFrame Head ---")
display(pl_df.tail(10))

# Display the results for the BS Sheet
print("\n--- BS DataFrame Columns ---")
print(bs_df.columns)
print("\n--- BS DataFrame Head ---")
display(bs_df.tail(10))


--- PL DataFrame Columns ---
MultiIndex([(                   '',        'Code'),
            (                   '', 'Description'),
            ('2025-06-01 00:00:00',      'Actual'),
            ('2025-06-01 00:00:00',      'Budget'),
            ('2025-06-01 00:00:00',    'Variance'),
            (                'YTD',      'Actual'),
            (                'YTD',      'Budget'),
            (                'YTD',    'Variance'),
            (                   '',  'Prior_Year'),
            (                   '',    'Variance')],
           )

--- PL DataFrame Head ---


C:\Users\georg\AppData\Local\Temp\ipykernel_25636\4039164483.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  top_header = top_header_raw.ffill().tolist()


\
      Code                                        Description   
142  Total                           Profit/(loss) before tax   
143      x                         Tax on ordinary activities   
144  Total               Profit/(loss) for the financial year   
145    NaN                                             Per TB   
146    NaN                                                NaN   
147    NaN                                                NaN   
148    NaN                                                NaN   
149    NaN                                                NaN   
150    NaN                                                NaN   
151    NaN  GWP of £3.8m was £3.9m below budget and suffer...   

    2025-06-01 00:00:00                                   YTD               \
                 Actual      Budget     Variance       Actual       Budget   
142        -2622.221480 -266.443126 -2355.778354 -3356.821320 -3225.474352   
143            0.000000    0.000000     0.000000     0.000000     0.000000   
144        -2622.221480 -266.443126 -2355.778354 -3356.821320 -3225.474352   
145            0.000000    0.000000     0.000000     0.000000     0.000000   
146            2.398342    0.676727    -1.721615     1.024783     0.697604   
147            0.000000    0.000000     0.000000     0.000000     0.000000   
148            0.152929    0.167408     0.014480     0.179899     0.171261   
149           -0.011955    0.022276     0.034231     0.012586     0.026352   
150            0.030241    0.005867    -0.024374     0.035338     0.006379   
151                 NaN         NaN          NaN          NaN          NaN   

                                           
       Variance   Prior_Year     Variance  
142 -131.346968 -8296.372560  4939.551240  
143    0.000000     0.000000     0.000000  
144 -131.346968 -8296.372560  4939.551240  
145    0.000000     0.000000     0.000000  
146   -0.327179     1.297194     0.272411  
147    0.000000    -0.006206    -0.006206  
148   -0.008637     0.181943     0.002045  
149    0.013766     0.037963     0.025377  
150   -0.028958     0.029015    -0.006323  
151         NaN          NaN          NaN


--- BS DataFrame Columns ---
MultiIndex([(                   '',        'Code'),
            (                   '', 'Description'),
            ('2025-06-01 00:00:00',      'Actual'),
            ('2025-06-01 00:00:00',      'Budget'),
            ('2025-06-01 00:00:00',    'Variance'),
            (                   '',  'Prior_Year'),
            (                   '',     'V to PY'),
            (                   '',            ''),
            (                   '',    'Comments')],
           )

--- BS DataFrame Head ---


2025-06-01 00:00:00                 \
        Code                 Description              Actual         Budget   
124     4261  Accruals Rescue and Excess           -0.218080      -0.219390   
125     4115             Trade Creditors            0.000010       0.000010   
126     4298             Legal Provision         -394.062460    -100.000000   
127      NaN                         NaN                 NaN       0.000000   
128  Heading                         NaN       -21518.301460  -34394.325964   
129  Heading           Total Liabilities      -281165.127030 -292419.310571   
130      NaN                         NaN            0.000000       0.000000   
131      NaN               Liquid assets        24290.583960   38158.754629   
132      NaN    Net Technical provisions       -40492.768750  -37995.775205   
133      NaN             Liquidity ratio            0.599875       1.004289   

                                                                              \
         Variance     Prior_Year       V to PY                                 
124      0.001310      -2.931390      2.713310                           NaN   
125      0.000000       0.000010      0.000000                           NaN   
126   -294.062460    -294.062460   -100.000000  Accruals and deferred income   
127           NaN            NaN           NaN                           NaN   
128  12876.024504  -53409.500040  31891.198580                           NaN   
129  11254.183541 -353843.040460  72677.913430                           NaN   
130           NaN       0.000000           NaN                           NaN   
131 -13868.170669   35279.769470 -10989.185510                           NaN   
132  -2496.993545  -46321.291650   5828.522900                           NaN   
133     -0.404415       0.761632     -0.161757                           NaN   

              
    Comments  
124      NaN  
125      NaN  
126      NaN  
127      NaN  
128      NaN  
129      NaN  
130      NaN  
131      NaN  
132      NaN  
133      NaN

In [20]:
def clean_financial_sheet(file_path, sheet_name):
    """
    Cleans a financial sheet (PL or BS) with a complex, multi-level header.

    Args:
        file_path (str): The path to the Excel file.
        sheet_name (str): The name of the sheet to clean ('PL' or 'BS').

    Returns:
        pandas.DataFrame: A cleaned DataFrame.
    """
    print(f"Processing sheet: {sheet_name}...")

    # Read the two header rows (rows 5 and 6, which are 0-indexed as 4 and 5)
    # and start the data from row 8 (skiprows=7)
    df = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=[4, 5],
        skiprows=2 # We will do a more targeted skip later, this is just to get the data loaded
    )

    # --- Header Cleaning ---
    # The columns are now a MultiIndex. Let's clean them up.
    new_cols = []
    # Forward-fill the top-level header (e.g., 'June 2025', 'YTD')
    top_level_header = pd.Series(df.columns.get_level_values(0)).ffill()
    bottom_level_header = df.columns.get_level_values(1)

    for i, (top, bottom) in enumerate(zip(top_level_header, bottom_level_header)):
        # Handle the first two descriptor columns which have no headers
        if 'Unnamed' in str(bottom):
            if i == 0:
                new_cols.append('Category')
            elif i == 1:
                new_cols.append('Description')
            else:
                 new_cols.append(bottom) # Keep other unnamed if they exist
        # For the main data columns, combine the two header levels
        else:
            # If the top level is a date or 'YTD', combine it with the bottom level
            if 'Unnamed' not in str(top):
                new_cols.append(f"{top.strftime('%B %Y') if hasattr(top, 'strftime') else top}_{bottom}")
            else:
                # For single-level headers like 'Prior_Year'
                new_cols.append(bottom)

    df.columns = new_cols

    # --- Body Cleaning ---
    # Find the actual start of the data (first row with a value in 'Category' or 'Description')
    first_data_row_index = df[df['Description'].notna() | df['Category'].notna()].index[0]
    df = df.iloc[first_data_row_index:].copy()


    # Find the end of the data (look for the row with lots of text in B, i.e., 'Description')
    # We assume any row where the description is longer than 100 chars is the footer text.
    footer_starts_at = df[df['Description'].str.len() > 100].index
    if not footer_starts_at.empty:
        df = df.loc[:footer_starts_at[0]-1]


    # Drop rows where ALL columns are empty
    df.dropna(how='all', inplace=True)

    # Drop any columns that are still entirely empty
    df.dropna(how='all', axis=1, inplace=True)

    # Reset the index to be clean and sequential
    df.reset_index(drop=True, inplace=True)

    print(f"Successfully cleaned sheet: {sheet_name}")
    return df


def clean_simple_sheet(file_path, sheet_name):
    """
    Cleans a simple sheet with a standard tabular layout.

    Args:
        file_path (str): The path to the Excel file.
        sheet_name (str): The name of the sheet to clean.

    Returns:
        pandas.DataFrame: A cleaned DataFrame.
    """
    print(f"Processing simple sheet: {sheet_name}...")
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=0)
    df.dropna(how='all', inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Successfully cleaned sheet: {sheet_name}")
    return df


# --- Configuration ---
# Replace with the path to your Excel file
excel_file_path = mi_file_path

# List the sheets with the complex financial layout
complex_sheets = ['PL', 'BS']
# List all other sheets that have a simple layout
simple_sheets = ['Deferrals']


# --- Execution ---
cleaned_dataframes = {}

# Process complex sheets
for sheet in complex_sheets:
    try:
        cleaned_dataframes[sheet] = clean_financial_sheet(excel_file_path, sheet)
    except Exception as e:
        print(f"ERROR: Could not process complex sheet '{sheet}'. Reason: {e}")

# Process simple sheets
for sheet in simple_sheets:
    try:
        cleaned_dataframes[sheet] = clean_simple_sheet(excel_file_path, sheet)
    except Exception as e:
        print(f"ERROR: Could not process simple sheet '{sheet}'. Reason: {e}")


# --- Display Results ---
# You can now access each cleaned dataframe by its sheet name
for sheet_name, df in cleaned_dataframes.items():
    print(f"\n--- First 5 rows of cleaned '{sheet_name}' sheet ---")
    print(df.head())
    print(f"\n--- Column names for '{sheet_name}' ---")
    print(df.columns.tolist())

Processing sheet: PL...
ERROR: Could not process complex sheet 'PL'. Reason: 'Description'
Processing sheet: BS...
ERROR: Could not process complex sheet 'BS'. Reason: 'Description'
Processing simple sheet: Deferrals...
Successfully cleaned sheet: Deferrals

--- First 5 rows of cleaned 'Deferrals' sheet ---
  Unnamed: 1                                        Unnamed: 2  Unnamed: 3  \
0     Act No                                          Act Name     2024.00   
1       3524               Deferred Excess of Loss Cover c/fwd  1247667.28   
2       3532  Reinsurers' Provision for Unearned Premiums - QS  4413421.05   
3       4852                        Provision unearned premium -6307147.26   
4       4152                     Deferred Profit Commission BS        0.00   

    Unnamed: 4   Unnamed: 6  
0      2025.00  Grand Total  
1   4163799.79   5411467.07  
2  16497285.31  20910706.36  
3 -23567550.47 -29874697.73  
4         0.00            0  

--- Column names for 'Deferrals' ---
['Un